<a href="https://colab.research.google.com/github/123rc/natual_language_processing/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import nltk
import re
from nltk.corpus import stopwords

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
train_data=pd.read_csv("/content/gdrive/MyDrive/fake-news/train.csv")
test_data=pd.read_csv("/content/gdrive/MyDrive/fake-news/test.csv")

In [5]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [7]:
train_data=train_data.dropna()



In [8]:
X=train_data.drop('label',axis=1)
y=train_data['label']

In [9]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
voc_size=5000

In [14]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4996, 3302, 2173, 2863, 4483, 2264, 4295, 4206, 1005, 4660],
 [4826, 2429, 1886, 207, 3580, 2102, 1252],
 [2104, 3067, 1166, 460],
 [2394, 136, 278, 657, 189, 2860],
 [3873, 3580, 135, 3358, 4650, 435, 3580, 4292, 3128, 4637],
 [1072,
  105,
  2257,
  3280,
  1655,
  1139,
  4960,
  4718,
  4756,
  3608,
  1960,
  4735,
  920,
  1026,
  1252],
 [4857, 4598, 1292, 1147, 2553, 3307, 2581, 3335, 1241, 301, 4342],
 [3190, 1344, 154, 2641, 4638, 3, 1139, 4987, 1241, 301, 4342],
 [2531, 3378, 891, 1390, 1953, 1713, 4977, 3872, 1139, 4831],
 [109, 468, 2293, 1854, 3347, 506, 1985, 3951],
 [3889, 232, 618, 4625, 68, 4367, 159, 735, 3830, 4847, 1297],
 [657, 392, 4483, 1713, 1139, 4638],
 [1087, 983, 3736, 1183, 1624, 2663, 2812, 3385, 3919],
 [4053, 63, 1985, 2287, 3280, 3411, 4346, 1241, 301, 4342],
 [4635, 1106, 3220, 117, 2050, 1241, 301, 4342],
 [3719, 4330, 1949, 1577, 4437, 3576, 475, 4047, 589, 2527],
 [553, 4439, 2429],
 [2879, 366, 1583, 3396, 1139, 384, 2986, 1252],
 [2011, 3651, 1

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4206 1005 4660]
 [   0    0    0 ... 3580 2102 1252]
 [   0    0    0 ... 3067 1166  460]
 ...
 [   0    0    0 ... 1241  301 4342]
 [   0    0    0 ... 1107 4902 3314]
 [   0    0    0 ...  711 3753  510]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4996,
       3302, 2173, 2863, 4483, 2264, 4295, 4206, 1005, 4660], dtype=int32)

In [21]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [23]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 45ms/step - loss: 0.3317 - accuracy: 0.8426 - val_loss: 0.1895 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 10s 54ms/step - loss: 0.1353 - accuracy: 0.9485 - val_loss: 0.1962 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0925 - accuracy: 0.9691 - val_loss: 0.2238 - val_accuracy: 0.9173
Epoch 4/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0641 - accuracy: 0.9791 - val_loss: 0.2698 - val_accuracy: 0.9087
Epoch 5/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0434 - accuracy: 0.9860 - val_loss: 0.2803 - val_accuracy: 0.9054
Epoch 6/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0307 - accuracy: 0.9913 - val_loss: 0.3601 - val_accuracy: 0.8988
Epoch 7/10
192/192 [==============================] - 11s 55ms/step - loss: 0.0184 - accuracy: 0.9943 - val_loss: 0.4269 - val_accuracy: 0.

In [28]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [39]:
y_pred=model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

189/189 [==============================] - 2s 9ms/step


In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917